In [7]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords # the for of in with
from nltk.stem.porter import PorterStemmer # loved loving == love
from sklearn.feature_extraction.text import TfidfVectorizer # loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
news_df = pd.read_csv('train.csv')

In [9]:
news_df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [10]:
news_df.shape

(72134, 4)

In [11]:
news_df.isna().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [12]:
news_df = news_df.fillna(' ')

In [13]:
news_df.isna().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [23]:
news_df['content'] = news_df['text']+" "+news_df['title']

In [20]:
news_df

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [24]:
print(news_df.columns)


Index(['Unnamed: 0', 'title', 'text', 'label', 'content'], dtype='object')


In [25]:
news_df['content']

0        No comment is expected from Barack Obama Membe...
1         Did they post their votes for Hillary already?  
2         Now, most of the demonstrators gathered last ...
3        A dozen politically active pastors came here f...
4        The RS-28 Sarmat missile, dubbed Satan 2, will...
                               ...                        
72129    WASHINGTON (Reuters) - Hackers believed to be ...
72130    You know, because in fantasyland Republicans n...
72131    Migrants Refuse To Leave Train At Refugee Camp...
72132    MEXICO CITY (Reuters) - Donald Trump’s combati...
72133    Goldman Sachs Endorses Hillary Clinton For Pre...
Name: content, Length: 72134, dtype: object

In [27]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [32]:
import nltk
nltk.download('stopwords')

'''from tqdm import tqdm
tqdm.pandas()

news_df['content'] = news_df['content'].progress_apply(stemming)'''


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"from tqdm import tqdm\ntqdm.pandas()\n\nnews_df['content'] = news_df['content'].progress_apply(stemming)"

In [36]:
from tqdm import tqdm
import pandas as pd

# Initialize tqdm for pandas
tqdm.pandas()

# Define your stemming function here
def stemming(text):
    # Example stemming logic (replace with your actual logic)
    return text.lower()  # Simple example of stemming

def process_chunk(chunk):
    # Apply the stemming function with progress tracking
    chunk['content'] = chunk['content'].progress_apply(stemming)
    return chunk

chunk_size = 1000  # Adjust based on your system's memory capacity
# Create chunks of the DataFrame
chunks = [news_df[i:i + chunk_size] for i in range(0, news_df.shape[0], chunk_size)]

processed_chunks = []

# Wrap the iteration with tqdm for a progress bar
for chunk in tqdm(chunks, desc="Processing Chunks"):
    processed_chunks.append(process_chunk(chunk))

# Concatenate all processed chunks back into a single DataFrame
news_df = pd.concat(processed_chunks, ignore_index=True)

100%|██████████| 1000/1000 [00:00<00:00, 10194.65it/s]/s]
C:\Users\kingo\AppData\Local\Temp\ipykernel_79988\716017473.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['content'] = chunk['content'].progress_apply(stemming)
Processing Chunks: 100%|██████████| 73/73 [00:07<00:00,  9.71it/s]


In [50]:
#news_df['content'] = news_df['content'].apply(stemming)

In [37]:
news_df['content']

0        no comment is expected from barack obama membe...
1         did they post their votes for hillary already?  
2         now, most of the demonstrators gathered last ...
3        a dozen politically active pastors came here f...
4        the rs-28 sarmat missile, dubbed satan 2, will...
                               ...                        
72129    washington (reuters) - hackers believed to be ...
72130    you know, because in fantasyland republicans n...
72131    migrants refuse to leave train at refugee camp...
72132    mexico city (reuters) - donald trump’s combati...
72133    goldman sachs endorses hillary clinton for pre...
Name: content, Length: 72134, dtype: object

In [38]:
X = news_df['content'].values
y = news_df['label'].values

In [39]:
print(X)

['no comment is expected from barack obama members of the #fyf911 or #fukyoflag and #blacklivesmatter movements called for the lynching and hanging of white people and cops. they encouraged others on a radio show tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in america.one of the f***yoflag organizers is called  sunshine.  she has a radio blog show hosted from texas called,  sunshine s f***ing opinion radio show. a snapshot of her #fyf911 @lolatwhitefear twitter page at 9:53 p.m. shows that she was urging supporters to  call now!! #fyf911 tonight we continue to dismantle the illusion of white below is a snapshot twitter radio call invite   #fyf911the radio show aired at 10:00 p.m. eastern standard time.during the show, callers clearly call for  lynching  and  killing  of white people.a 2:39 minute clip from the radio show can be heard here. it was provided to breitbart texas by someone who would like to be referred t

In [40]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [41]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 18697115 stored elements and shape (72134, 244445)>
  Coords	Values
  (0, 0)	0.028754657559453688
  (0, 702)	0.01514038493339824
  (0, 1113)	0.058764278439465925
  (0, 1133)	0.03272001544748171
  (0, 6863)	0.02719780921590389
  (0, 7809)	0.02766015739548307
  (0, 11399)	0.008039579175805557
  (0, 11918)	0.02664793015402819
  (0, 12445)	0.01607833858155022
  (0, 13637)	0.050535248234163575
  (0, 13742)	0.014877720125122356
  (0, 13745)	0.010893290416752114
  (0, 14376)	0.06639273567371945
  (0, 14440)	0.030332602541828364
  (0, 15338)	0.02844638435420013
  (0, 15737)	0.008850392625682735
  (0, 15795)	0.024121940401104467
  (0, 16138)	0.020962642941426864
  (0, 16224)	0.04438960070472016
  (0, 16568)	0.017814233138895314
  (0, 16892)	0.013342723006182703
  (0, 16911)	0.02432688839564493
  (0, 17373)	0.007095596764324894
  (0, 17593)	0.12023633042335953
  (0, 18416)	0.012824722860673366
  :	:
  (72133, 195592)	0.111136534955317

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [43]:
X_train.shape

(57707, 244445)

In [44]:
X_test.shape

(14427, 244445)

In [45]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [46]:
train_y_pred = model.predict(X_train)
print("train accurracy :",accuracy_score(train_y_pred,y_train))

train accurracy : 0.9654981198121545


In [47]:
test_y_pred = model.predict(X_test)
print("train accurracy :",accuracy_score(test_y_pred,y_test))

train accurracy : 0.9480141401538782


In [48]:
# prediction system

input_data = X_test[20]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Fake news


In [49]:
news_df['content'][20]

'wednesday 9 november 2016 by lucas wilde america gives grand piano to horse \namerica has given a grand piano to a horse and is expecting some quality tunes. \n“i’m particularly looking forward to beethoven’s ninth,” beamed horse supporter and piano enthusiast, jay cooper. \n“a horse has never been given a piano before because, frankly, the establishment wouldn’t allow it. \n“now, at last, change has come, and america will change for the better. \n“there are a lot of doubters out there, and those doubters will soon be silenced by the graceful notes of chopin, mozart and maybe even little richard.” \nhorse, dobbin williams, said, “i’m not really sure what’s expected of me here. \n“i’m a horse. i am absolutely not qualified to play a piano. \n“i mean… look at these hooves and the way i am in general. i can’t even sit on the chair properly. \n“why on earth did anyone think this was a good idea?” \ncooper grinned, “we did it. we’ve made pianos great again.” \ndemocrat, elizabeth king, sai